**Importing the dependencies**

In [ ]:
import os
import json
import requests

from azureml.core import Workspace
from azureml.core.model import Model
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice, Webservice, LocalWebservice

In [ ]:
# loading the configuration file - standard way - use .env file and load_dotenv from python-dotenv module
config_file_path = "config.json"

# Read JSON data into a dictionary
with open(config_file_path, 'r') as file:
    data = json.load(file)

subscription_id = data["subscription_id"]
resource_group = data["resource_group"]  
workspace_name = data["workspace_name"]
region = data["region"]
model_name = data["model_name"]
service_name = data["service_name"]
model_path = data["weights_path"]

In [ ]:
print(f"Resource group: {resource_group}")
print(f"Workspace: {workspace_name}")
print(f"Region: {region}")
print(f"Model Name: {model_name}")
print(f"Service Name: {service_name}")
print(f"Model Path: {model_path}")

**Create a Resource Group from Azure Portal**

In [ ]:
# Create a workspace

try:
    ws = Workspace.get(name=workspace_name,
                        subscription_id=subscription_id,
                        resource_group=resource_group,
                        location=region)
    print(f'Workspace {workspace_name} loaded')
except:
    ws = Workspace.create(name=workspace_name,
                        subscription_id=subscription_id,
                        resource_group=resource_group,
                        location=region)
    print(f'Workspace {workspace_name} created')

In [ ]:
# Register the model in Azure Machine Learning
try:
    registered_model = Model(workspace=ws, name=model_name)
    print(f"{model_name} loaded from registry")
except:
    registered_model = Model.register(model_path=model_path, model_name=model_name, workspace=ws)
    print(f"{model_name} registred")

In [ ]:
conda_env = Environment('az2')

In [ ]:
ymlenv = Environment.from_conda_specification(name="myenv", file_path="environment.yml")

In [ ]:
# Create an InferenceConfig
inference_config = InferenceConfig(entry_script='score.py', environment=ymlenv)

In [ ]:
local_deployment = False

In [ ]:
# Specify deployment configuration for ACI
if local_deployment == True:
    aci_config = LocalWebservice.deploy_configuration()
else:
    aci_config = AciWebservice.deploy_configuration(cpu_cores=2, memory_gb=2)

In [ ]:
def delete_service():
    service = ws.webservices[service_name]
    service.delete()
    print(f"{service_name} sucessfully deleted.")

In [ ]:
try:
    delete_service()
except:
    print("No Service exists")

In [ ]:
service = Model.deploy(workspace=ws,
                       name=service_name,
                       models=[registered_model],
                       inference_config=inference_config,
                       deployment_config=aci_config)
service.wait_for_deployment(show_output=True)

In [ ]:
print(service.get_logs())

In [ ]:
scoring_uri = service.scoring_uri

In [ ]:
scoring_uri

**Cleaning up all the created resources**

In [ ]:
delete_service()

In [ ]:
# Delete the model
"""
registered_model = ws.models[model_name]
registered_model.delete()
print(f"{model_name} sucessfully deleted.")
"""


In [ ]:
# Delete the workspace
"""
ws.delete(delete_dependent_resources=True)
print(f"{workspace_name} sucessfully deleted.")
"""

1. Azure Account with appropriate subscription
2. Configuration file that has subscription_id and other details like resource_group name, etc
3. Create a resource group from Azure Portal
4. Create a workspace
5. Register the model in container registry
6. Deploy the model as endpoint
7. Once done, delete the created resources